# CDR-Plugin-Folder-to-Folder

# Step 0 : Install Pre-requisites

**Wait till all packages get installed**


In [15]:
import sys

!{sys.executable} -m conda install git -y
!{sys.executable} -m pip install urllib3
!{sys.executable} -m pip install git+https://github.com/owasp-sbot/OSBot-Utils.git
print("Pre-requisites installations are completed")

Solving environment: done

# All requested packages already installed.

  Cloning https://github.com/owasp-sbot/OSBot-Utils.git to /tmp/pip-req-build-p2u86hgs
  Running command git clone -q https://github.com/owasp-sbot/OSBot-Utils.git /tmp/pip-req-build-p2u86hgs
Pre-requisites installations are completed


# Step 1 :  Setup Client

In [46]:
import os
from urllib.parse import urljoin
from osbot_utils.utils.Http import GET_json,POST,POST_json

class Client:

    def __init__(self, url_server):
        self.server_ip = url_server

    # helper methods
    def _resolve_url(self, path=""):
        return urljoin(self.server_ip, path)

    def _request_get(self, path):
        url = self._resolve_url(path)
        return GET_json(url)

    def _request_post(self, path):
        url = self._resolve_url(path)
        return POST(url=url,data=b'', headers=None)

    # API methods
    def health(self):
        return self._request_get('/health')

    def version(self):
        return self._request_get('/version')

    def pre_process(self):
        return self._request_post('/pre-processor/pre-process')

    def start_process(self):
        return self._request_post('/processing/start')
    
    def show_folder_structure(self, source_path):
        response=""
        for root, dirs, files in os.walk(source_path):
            level = root.replace(source_path, '').count(os.sep)
            indent = ' ' * 4 * (level)
            print('{}{}/'.format(indent, os.path.basename(root)))
            subindent = ' ' * 4 * (level + 1)
            for f in files:
                print('{}{}'.format(subindent, f))
                
            return response

# Step 2 :  Get API Endpoint

In [47]:
api_url='http://api:8880'
client=Client(api_url)

# Step 3 : Health checlk

### Status should be **{'status': 'ok'}**

In [43]:
response=client.health()
print(response)

{'status': 'ok'}


# Step 4 : Pre-Processing
### This api will pre-process HD1 data and store it in HD2

**Response should be ["Processing is done"]**

In [39]:
response=client.pre_process()
print(response)

["Processing is done"]


# Step 5 : Processing
### This api will process HD2 data and store rebuilt file in HD3

**Response should be "Loop completed"**

In [51]:
response=client.start_process()
print(response)

# Step 6 : Folder structure visualization

In [58]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))
    
paths = DisplayablePath.make_tree(Path('../test_data/scenario-1'))
for path in paths:
    print(path.displayable())

scenario-1/
├── hd1/
│   └── image2.jpg
├── hd2/
│   ├── data/
│   │   └── 86df34018436a99e1e98ff51346591e189cd76c8518f7288bb1ea8336396259b/
│   │       ├── metadata.json
│   │       ├── report.json
│   │       └── source
│   └── status/
│       └── hash.json
└── hd3/
    └── image2.jpg
